<a href="https://colab.research.google.com/github/syedmehrabzaidi/AI-LAB/blob/main/RAG_complete_flow_with_document_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install necessary packages
!pip install -q faiss-cpu==1.7.4 sentence-transformers==2.2.2 langchain openai python-dotenv



In [ ]:
!pip install numpy==1.26.4

Explanation:

This cell installs the necessary libraries and imports them. Faiss is used for efficient similarity search and SentenceTransformer is used for generating sentence embeddings. **bold text**

In [ ]:
!pip install -U langchain-community==0.2.12

In [ ]:
import os
import faiss
import numpy as np
import pandas as pd
from getpass import getpass
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.callbacks import StdOutCallbackHandler
from google.colab import files

In [ ]:

# Cell 2: Set up environment variables
# Load environment variables from .env file if it exists
load_dotenv()

# Get OpenAI API key - either from environment or prompt user
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    openai_api_key = getpass("Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

print("Environment setup complete!")

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
!pip install pypdf

**process_documents()**

In [ ]:
#  Upload and process documents, then create embeddings
import time
import gc
import pickle

def process_documents():
    # Create temporary directories
    !mkdir -p ./temp_docs
    !mkdir -p ./pickle

    # Allow user to upload documents
    uploaded = files.upload()

    documents = []

    # Save uploaded files and process them
    for filename, content in uploaded.items():
        file_path = f"./temp_docs/{filename}"
        with open(file_path, 'wb') as f:
            f.write(content)

        print(f"Processing: {filename}")

        # Load documents based on file type
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())
        elif filename.endswith('.txt'):
            loader = TextLoader(file_path)
            documents.extend(loader.load())
        else:
            print(f"Unsupported file type: {filename}")

    if not documents:
        print("No documents were successfully loaded.")
        return None

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split documents into {len(chunks)} chunks")

    # Create FAISS index with embeddings
    vectorstore = FAISS.from_documents(chunks, embeddings)
    print("FAISS index created successfully!")

    # Generate a unique db_id for this session
    db_id = f"session_{int(time.time())}"

    # Save the retriever to pickle file
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )

    gc.disable()
    with open(f'./pickle/123.pkl', 'wb') as f:
        pickle.dump(retriever, f)
    gc.enable()

    print(f"Retriever saved with db_id: 123.pkl")

    return db_id, vectorstore

# Run the document processing function
db_id, vectorstore = process_documents()

Saving talha.pdf to talha.pdf
Processing: talha.pdf
Split documents into 1 chunks


NameError: name 'embeddings' is not defined

**3# Upload and process documents**

In [ ]:
# #  create embeddings

# def process_documents():
#     # Create a temporary directory to store uploaded files
#     !mkdir -p ./temp_docs

#     # Allow user to upload documents
#     uploaded = files.upload()

#     documents = []

#     # Save uploaded files and process them
#     for filename, content in uploaded.items():
#         file_path = f"./temp_docs/{filename}"
#         with open(file_path, 'wb') as f:
#             f.write(content)

#         print(f"Processing: {filename}")

#         # Load documents based on file type
#         if filename.endswith('.pdf'):
#             loader = PyPDFLoader(file_path)
#             documents.extend(loader.load())
#         elif filename.endswith('.txt'):
#             loader = TextLoader(file_path)
#             documents.extend(loader.load())
#         else:
#             print(f"Unsupported file type: {filename}")

#     if not documents:
#         print("No documents were successfully loaded.")
#         return None

#     # Split documents into chunks
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200
#     )
#     chunks = text_splitter.split_documents(documents)
#     print(f"Split documents into {len(chunks)} chunks")

#     # Create FAISS index with embeddings
#     vectorstore = FAISS.from_documents(chunks, embeddings)
#     print("FAISS index created successfully!")

#     return vectorstore

# # Run the document processing function
# vectorstore = process_documents()

Saving cricket.pdf to cricket (2).pdf
Processing: cricket (2).pdf
Split documents into 1 chunks
FAISS index created successfully!


In [ ]:
# # Cell 4: Set up history-aware retriever
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.prompts import MessagesPlaceholder
# from langchain.chains import create_history_aware_retriever
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain

# # from langchain.chat_models import ChatOpenAI


# def setup_retriever(db_id):
#     if not db_id:
#         print("No db_id available. Please upload and process documents first.")
#         return None, None


#     from google.colab import userdata

#     # Get the API key from user data
#     api_key = userdata.get('OPENAI_API_KEY')

#     # Print the API key (for verification) - optional
#     print(f"API Key-------------------------------------------: loaded")
#     # Initialize LLM
#     llm_src = ChatOpenAI(
#         temperature=float(0.1),
#         openai_api_key=api_key,
#         model="gpt-3.5-turbo"
#     )

#     try:
#         # Load the retriever from pickle file
#         gc.disable()
#         with open(f'./pickle/abc_main.pkl', 'rb') as f:
#             retriever = pickle.load(f)
#         gc.enable()

#         # Create the prompt for generating search queries
#         prompt_search_query = ChatPromptTemplate.from_messages([
#             MessagesPlaceholder(variable_name="chat_history"),
#             ("user", "{input}"),
#             ("user", "Given the above conversation, generate a search query to look up information relevant to the conversation")
#         ])

#         # Create history-aware retriever
#         history_aware_retriever = create_history_aware_retriever(
#             llm_src,
#             retriever,
#             prompt_search_query
#         )

#         # Set up the prompt template for generating answers
#         prompt_get_answer = ChatPromptTemplate.from_messages([
#             ("system", "Answer the user's questions based on the below context:\n\n{context}"),
#             ("system", "{systemPrompt}"),  # Insert the custom system prompt here
#             MessagesPlaceholder(variable_name="chat_history"),
#             ("user", "{input}"),
#         ])

#         # Create the document chain
#         document_chain = create_stuff_documents_chain(llm_src, prompt_get_answer)

#         # Create the retrieval chain
#         retrieval_chain = create_retrieval_chain(history_aware_retriever, document_chain)

#         print("History-aware retriever set up successfully!")
#         return retrieval_chain, retriever

#     except FileNotFoundError:
#         print(f"Data with db_id {db_id} does not exist.")
#         return None, None

# # Set up the retriever
# retrieval_chain, retriever = setup_retriever(db_id)

API Key-------------------------------------------: loaded
Data with db_id session_1744790701 does not exist.


In [ ]:
# # Cell 5: Chat interface with history tracking
# from langchain_core.messages import HumanMessage
# from langchain.schema import SystemMessage


# def chat_with_documents(system_prompt="You are a helpful assistant."):
#     if retrieval_chain is None:
#         print("Please set up the retriever chain first.")
#         return

#     # Initialize chat history
#     chat_history = []

#     print("Chat with your documents! Type 'exit' to end the conversation.")
#     print("-------------------------------------------------------------")

#     while True:
#         question = input("You: ")

#         if question.lower() in ['exit', 'quit', 'bye']:
#             print("Goodbye!")
#             break

#         # Format chat history for the chain
#         formatted_chat_history = []
#         for item in chat_history:
#             formatted_chat_history.append(HumanMessage(content=item["question"]))
#             formatted_chat_history.append(SystemMessage(content=item["answer"]))

#         # Add the current question to history for processing
#         current_chat_history = formatted_chat_history.copy()

#         print("------current_chat_history----------------------------------------",current_chat_history)

#         # Process the query through the retrieval chain
#         response = retrieval_chain.invoke({
#             "chat_history": current_chat_history,
#             "input": question,
#             "systemPrompt": system_prompt
#         })

#         answer = response['answer']

#         # Display the answer
#         print(f"Assistant: {answer}")

#         # Append to chat history
#         chat_history.append({"question": question, "answer": answer})

#         # Optionally display relevant documents (similar to your reference code)
#         if retriever:
#             print("\n-----Relevant Document Snippets-----")
#             documents = retriever.get_relevant_documents(question)
#             if len(documents) >= 1:
#                 print(f"Document 1: {documents[0].page_content[:150]}...")
#             if len(documents) >= 2:
#                 print(f"Document 2: {documents[1].page_content[:150]}...")
#             print("-----------------------------------\n")

#     # Return the chat history
#     return chat_history

# # Create a custom system prompt
# custom_system_prompt = """You are a helpful AI assistant that answers questions based on the provided documents.
# Be concise, accurate, and only use information from the context when possible."""

# # Run the chat interface
# if retrieval_chain:
#     history = chat_with_documents(system_prompt=custom_system_prompt)

#     # Display the full conversation history
#     if history:
#         print("\nConversation History:")
#         for i, exchange in enumerate(history, 1):
#             print(f"\nExchange {i}:")
#             print(f"Question: {exchange['question']}")
#             print(f"Answer: {exchange['answer']}")

In [ ]:
#-------------------------------------------------

**4# retriever**

In [ ]:
# history-aware retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Global variable declaration
global retrieval_chain
global retriever

def setup_retriever(db_id):
    global retrieval_chain
    global retriever

    from google.colab import userdata

    # Get the API key from user data
    api_key = userdata.get('OPENAI_API_KEY')

    # Print verification message
    print(f"API Key: loaded")

    # Initialize LLM
    llm_src = ChatOpenAI(
        temperature=float(0.1),
        openai_api_key=api_key,
        model="gpt-3.5-turbo"
    )

    try:
        # Try loading the retriever from pickle file
        gc.disable()
        try:
            with open(f'./pickle/123.pkl', 'rb') as f:
                retriever = pickle.load(f)
            print(f"Successfully loaded existing retriever from pickle/123_main.pkl")
        except FileNotFoundError:
            # If file not found, try the hardcoded path as a fallback
            try:
                with open('./pickle/123.pkl', 'rb') as f:
                    retriever = pickle.load(f)
                print("Successfully loaded existing retriever from pickle/abc_main.pkl")
            except FileNotFoundError:
                # If both fail, create a new retriever from scratch
                print("No existing retriever found. Creating a new one...")

                # Check if we have documents already processed
                if 'documents' in globals() and documents:
                    print("Using previously processed documents")
                    chunks = documents
                else:
                    # If no documents, create a sample document for functionality testing
                    print("No documents found. Creating a sample document for testing...")
                    sample_text = """This is a sample document created because no existing documents or retriever were found.
                    The system will work better when you upload your own documents.
                    You can upload PDF or text files in Cell 3 to use your own content."""

                    # Create a document loader for the sample text
                    with open('./temp_docs/sample.txt', 'w') as f:
                        f.write(sample_text)

                    from langchain.document_loaders import TextLoader
                    loader = TextLoader('./temp_docs/sample.txt')
                    documents = loader.load()

                    # Split the document into chunks
                    text_splitter = RecursiveCharacterTextSplitter(
                        chunk_size=1000,
                        chunk_overlap=200
                    )
                    chunks = text_splitter.split_documents(documents)

                # Initialize embeddings and create vector store
                embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
                vectorstore = FAISS.from_documents(chunks, embeddings)

                # Create retriever from vector store
                retriever = vectorstore.as_retriever(
                    search_type="similarity",
                    search_kwargs={"k": 5}
                )

                # Save the new retriever to pickle
                os.makedirs('./pickle', exist_ok=True)
                with open(f'./pickle/123.pkl', 'wb') as f:
                    pickle.dump(retriever, f)
                print(f"Created and saved new retriever to pickle/123.pkl")

        gc.enable()

        # Create the prompt for generating search queries
        prompt_search_query = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            ("user", "Given the above conversation, generate a search query to look up information relevant to the conversation")
        ])

        # Create history-aware retriever
        history_aware_retriever = create_history_aware_retriever(
            llm_src,
            retriever,
            prompt_search_query
        )

        # Set up the prompt template for generating answers
        prompt_get_answer = ChatPromptTemplate.from_messages([
            ("system", "Answer the user's questions based on the below context:\n\n{context}"),
            ("system", "{systemPrompt}"),  # Insert the custom system prompt here
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ])

        # Create the document chain
        document_chain = create_stuff_documents_chain(llm_src, prompt_get_answer)

        # Create the retrieval chain
        retrieval_chain = create_retrieval_chain(history_aware_retriever, document_chain)

        print("History-aware retriever set up successfully!")
        return retrieval_chain, retriever

    except Exception as e:
        print(f"Error setting up retriever: {str(e)}")
        return None, None

# Initialize variables before use
retrieval_chain = None
retriever = None

# If db_id isn't defined, create a default one
if 'db_id' not in globals() or db_id is None:
    db_id = "default_session"
    print(f"No db_id found, using '{db_id}' as default")

# Set up the retriever
retrieval_chain, retriever = setup_retriever("123")

# Verification
if retrieval_chain is not None:
    print("Retrieval chain is ready to use")
else:
    print("Warning: Failed to set up retrieval chain")

API Key: loaded
Successfully loaded existing retriever from pickle/123_main.pkl
History-aware retriever set up successfully!
Retrieval chain is ready to use


**5# CHAT**

In [ ]:
# Cell 5: Chat interface with history tracking
from langchain_core.messages import HumanMessage
from langchain.schema import SystemMessage


# Access global variables
# global retrieval_chain
# global retriever

def chat_with_documents(system_prompt="You are a helpful assistant."):
    # Access global variables inside function
    global retrieval_chain
    global retriever

    if retrieval_chain is None:
        print("Please set up the retriever chain first.")
        return

    # Initialize chat history
    chat_history = []

    print("Chat with your documents! Type 'exit' to end the conversation.")
    print("-------------------------------------------------------------")

    while True:
        question = input("You: ")

        if question.lower() in ['exit', 'quit', 'bye']:
            print("Goodbye!")
            break

        # Format chat history for the chain
        formatted_chat_history = []
        for item in chat_history:
            formatted_chat_history.append(HumanMessage(content=item["question"]))
            formatted_chat_history.append(SystemMessage(content=item["answer"]))

        # Add the current question to history for processing
        current_chat_history = formatted_chat_history.copy()

        try:
            # Process the query through the retrieval chain
            response = retrieval_chain.invoke({
                "chat_history": current_chat_history,
                "input": question,
                "systemPrompt": system_prompt
            })

            answer = response['answer']

            # Display the answer
            print(f"Assistant: {answer}")

            # Append to chat history
            chat_history.append({"question": question, "answer": answer})

            # Display relevant documents
            if retriever:
                print("\n-----Relevant Document Snippets-----")
                documents = retriever.get_relevant_documents(question)
                if len(documents) >= 1:
                    print(f"Document 1: {documents[0].page_content[:150]}...")
                if len(documents) >= 2:
                    print(f"Document 2: {documents[1].page_content[:150]}...")
                print("-----------------------------------\n")

        except Exception as e:
            print(f"\nError processing query: {str(e)}")
            print("Please try asking another question or restart the notebook if the issue persists.")

    # Return the chat history
    return chat_history

# Create a custom system prompt
custom_system_prompt = """You are a helpful AI assistant that answers questions based on the provided documents.
Be concise, accurate, and only use information from the context when possible."""

# Run the chat interface with proper global variable checking
if 'retrieval_chain' in globals() and retrieval_chain is not None:
    history = chat_with_documents(system_prompt=custom_system_prompt)

    # Display the full conversation history
    if history:
        print("\nConversation History:")
        for i, exchange in enumerate(history, 1):
            print(f"\nExchange {i}:")
            print(f"Question: {exchange['question']}")
            print(f"Answer: {exchange['answer']}")
else:
    print("Error: retrieval_chain is not properly defined. Please run the previous cells first.")

Chat with your documents! Type 'exit' to end the conversation.
-------------------------------------------------------------
Assistant: Markus Schranner is the Chief Executive Officer & Co-Founder of heronOS, bringing leadership experience in the tech industry with a focus on AI-driven innovation and organizational growth.

-----Relevant Document Snippets-----
Document 1: HeronOS was founded by a team of visionary leaders with extensive expertise in AI 
technology, business development, and digital transformation.
- **M...
Document 2: - **Christoph Freyermuth**  
  **Chief Product Officer & Co-Founder**  
  Christoph focuses on product development, working to ensure that heronOS’s A...
-----------------------------------

Assistant: Ralph Stenzel is the Chief Technical Officer & Co-Founder of heronOS. He leads the development of heronOS's cutting-edge AI solutions to ensure the product remains at the forefront of technological advancements.

-----Relevant Document Snippets-----
Documen

KeyboardInterrupt: Interrupted by user